In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

df = pd.read_csv('googleplaystore.csv.csv')

now = datetime.now()

if now.hour >= 18 and now.hour < 21:

    # Apply all filters
    df = df[~df['App'].str[0].str.upper().isin(['X', 'Y', 'Z'])]
    df = df[df['Category'].str[0].isin(['E', 'C', 'B'])]
    df = df[df['Reviews'] > 500]
    df = df[~df['App'].str.contains('S', case=True, na=False)]

    # Get month from date
    df['Month'] = pd.to_datetime(df['Last Updated']).dt.to_period('M').astype(str)

    # Group and calculate growth
    data = df.groupby(['Month', 'Category'])['Installs'].sum().reset_index()
    data = data.sort_values(['Category', 'Month'])
    data['Prev'] = data.groupby('Category')['Installs'].shift(1)
    data['Growth'] = ((data['Installs'] - data['Prev']) / data['Prev']) * 100

    # Translations and colors
    names = {'Beauty': 'सौन्दर्य', 'Business': 'வியிர்ஏஷ்ஷணம்', 'Dating': 'Verabredung'}
    colors = {'Entertainment': 'blue', 'Communication': 'green', 'Business': 'orange',
              'Beauty': 'red', 'Comics': 'purple', 'Dating': 'brown', 'Events': 'teal'}

    fig = go.Figure()

    for cat in data['Category'].unique():
        cd = data[data['Category'] == cat].reset_index(drop=True)
        name = names.get(cat, cat)
        color = colors.get(cat, 'gray')

        # Add main line
        fig.add_trace(go.Scatter(
            x=cd['Month'], y=cd['Installs'], name=name,
            mode='lines+markers',
            line=dict(color=color, width=2),
            marker=dict(size=6)
        ))

        # Shade 20%+ growth areas
        for i in range(1, len(cd)):
            if cd['Growth'][i] > 20:
                fig.add_trace(go.Scatter(
                    x=[cd['Month'][i-1], cd['Month'][i]],
                    y=[cd['Installs'][i-1], cd['Installs'][i]],
                    mode='lines', line=dict(width=0),
                    fill='tonexty', fillcolor=color,
                    opacity=0.2, showlegend=False, hoverinfo='skip'
                ))

    fig.update_layout(
        title='Total Installs Over Time (Shaded = 20%+ Growth)',
        xaxis=dict(title='Month', tickangle=-45),
        yaxis=dict(title='Total Installs', tickformat=','),
        height=650, showlegend=True
    )

    fig.show()

else:
    print(f"Only available between 6 PM - 9 PM IST. Current: {now.strftime('%I:%M %p')}")

Only available between 6 PM - 9 PM IST. Current: 09:13 AM
